# 단일 ETF, 시장 데이터를 이용한 이벤트 스터디

In [261]:
import pandas as pd
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

# 주가 데이터 로드 (예: CSV 파일에서)
df = pd.read_csv('C:/Users/swc08/Desktop/대학/DB보험금융공모전/데이터셋/ESGU.csv', parse_dates=['Date'])
df.set_index('Date', inplace=True)

# 이벤트 날짜와 이벤트 윈도우, 추정 윈도우 설정
event_date = pd.Timestamp('2017-01-20')
estimation_window = 30  # 추정 기간: 이벤트 이전 30일
event_window = 10       # 이벤트 기간: 이벤트 전후 10일

# 이벤트 윈도우와 추정 윈도우 설정
estimation_start = event_date - pd.Timedelta(days=estimation_window + event_window)
estimation_end = event_date - pd.Timedelta(days=1)
event_start = event_date - pd.Timedelta(days=event_window)
event_end = event_date + pd.Timedelta(days=event_window)




In [262]:
df1 = pd.read_csv('C:/Users/swc08/Desktop/대학/DB보험금융공모전/데이터셋/ESGU.csv', parse_dates=['Date'])
df1.set_index('Date', inplace=True)

In [263]:
df2 = pd.read_csv('C:/Users/swc08/Desktop/대학/DB보험금융공모전/데이터셋/SPY.csv', parse_dates=['Date'])
df2.set_index('Date', inplace=True)

In [264]:
df3 = pd.concat([df1['Close'], df2['Close']],axis=1)

In [265]:
df3.columns = [['Stock_Price', 'Market_Price']]

In [266]:
# 추정 윈도우 데이터 추출
estimation_data = df3.loc[estimation_start:estimation_end]
event_data = df3.loc[event_start:event_end]

In [267]:
import numpy as np
event_data["Stock_Return"] = np.log(event_data["Stock_Price"] / event_data["Stock_Price"].shift(1))
event_data["Market_Return"] = np.log(event_data["Market_Price"] / event_data["Market_Price"].shift(1))

In [268]:

# 시장 수익률과 개별 주식 수익률 계산
estimation_data['Market_Return'] = estimation_data['Market_Price'].pct_change()
estimation_data['Stock_Return'] = estimation_data['Stock_Price'].pct_change()

# 회귀 분석을 통한 알파와 베타 추정
X = sm.add_constant(estimation_data['Market_Return'].dropna())
y = estimation_data['Stock_Return'].dropna()
model = sm.OLS(y, X).fit()
alpha, beta = model.params


In [269]:
event_data['Expected_Return'] = (alpha + beta * event_data['Market_Return'])
event_data

,Stock_Price,Market_Price,Stock_Return,Market_Return,Expected_Return
Date,,,,,
2017-01-10,44.065697,198.357468,NaN,NaN,NaN
2017-01-11,44.065697,198.918045,0.000000,0.002822,0.002711
2017-01-12,44.065697,198.418732,0.000000,-0.002513,-0.000574
2017-01-13,44.065697,198.874237,0.000000,0.002293,0.002385
2017-01-17,43.897419,198.173508,-0.003826,-0.003530,-0.001200
2017-01-18,43.897419,198.611496,0.000000,0.002208,0.002333
2017-01-19,43.897419,197.875732,0.000000,-0.003711,-0.001312
2017-01-20,43.897419,198.602737,0.000000,0.003667,0.003232
2017-01-23,43.897419,198.085953,0.000000,-0.002605,-0.000631


In [270]:
a = event_data['Stock_Return'].values
b = event_data['Expected_Return'].values

In [271]:
event_data['Abnormal_Return'] = a-b
event_data['CAR'] = event_data['Abnormal_Return'].cumsum()


event_data[['Stock_Return', 'Expected_Return', 'Abnormal_Return', 'CAR']]

,Stock_Return,Expected_Return,Abnormal_Return,CAR
Date,,,,
2017-01-10,NaN,NaN,NaN,NaN
2017-01-11,0.000000,0.002711,-0.002711,-0.002711
2017-01-12,0.000000,-0.000574,0.000574,-0.002137
2017-01-13,0.000000,0.002385,-0.002385,-0.004522
2017-01-17,-0.003826,-0.001200,-0.002626,-0.007149
2017-01-18,0.000000,0.002333,-0.002333,-0.009481
2017-01-19,0.000000,-0.001312,0.001312,-0.008170
2017-01-20,0.000000,0.003232,-0.003232,-0.011401
2017-01-23,0.000000,-0.000631,0.000631,-0.010770


# 다중 ETF, 시장 데이터를 이용한 이벤트 스터디

In [114]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

# ESG ETF 데이터 불러오기
esgu_etf = pd.read_csv('C:/Users/swc08/Desktop/대학/DB보험금융공모전/데이터셋/ESGU.csv', parse_dates=['Date'])
esgv_etf = pd.read_csv('C:/Users/swc08/Desktop/대학/DB보험금융공모전/데이터셋/ESGV.csv', parse_dates=['Date'])
vde_etf = pd.read_csv('C:/Users/swc08/Desktop/대학/DB보험금융공모전/데이터셋/VDE.csv', parse_dates=['Date'])
sdus_etf = pd.read_csv('C:/Users/swc08/Desktop/대학/DB보험금융공모전/데이터셋/SDUS.csv', parse_dates=['Date'])
tan_etf = pd.read_csv('C:/Users/swc08/Desktop/대학/DB보험금융공모전/데이터셋/TAN.csv', parse_dates=['Date'])
evus_etf = pd.read_csv('C:/Users/swc08/Desktop/대학/DB보험금융공모전/데이터셋/EVUS.csv', parse_dates=['Date'])

esgu_etf.set_index('Date', inplace=True)
esgv_etf.set_index('Date', inplace=True)
vde_etf.set_index('Date', inplace=True)
sdus_etf.set_index('Date', inplace=True)
tan_etf.set_index('Date', inplace=True)
evus_etf.set_index('Date', inplace=True)


# 시장 데이터 불러오기
spy_etf = pd.read_csv('C:/Users/swc08/Desktop/대학/DB보험금융공모전/데이터셋/SPY.csv', parse_dates=['Date'])
spy_etf.set_index('Date', inplace=True)


In [115]:
# 이벤트 날짜 (가정 : 모든 ETF 동일)
event_date = pd.Timestamp("2024-01-20")

# 이벤트 윈도우 설정
estimation_window = 30  # 이벤트 이전 30일
event_window = 10  # 이벤트 전후 10일

In [116]:
df = pd.concat([esgu_etf['Close'],
                esgv_etf['Close'],
                vde_etf['Close'],
                sdus_etf['Close'],
                tan_etf['Close'],
                evus_etf['Close'],
                spy_etf['Close']],axis=1)

In [117]:
df.columns = [['esgu_etf','esgv_etf',
               'vde_etf','sdus_etf',
               'tan_etf','evus_etf',
               'spy_etf']]

In [118]:
etf_list = []
for i in range(0,7):
    etf_list.append(df.columns[i][0])

In [119]:
# 윈도우 설정
estimation_start = event_date - pd.Timedelta(days=estimation_window + event_window)
estimation_end = event_date - pd.Timedelta(days=1)
event_start = event_date - pd.Timedelta(days=event_window)
event_end = event_date + pd.Timedelta(days=event_window)

# 추정 윈도우 및 이벤트 윈도우 데이터 추출
estimation_data = df.loc[estimation_start:estimation_end]
event_data = df.loc[event_start:event_end]

event_data = event_data.dropna()
estimation_data = event_data.dropna()

In [120]:
event_data["Market_Return"] = np.log(event_data["spy_etf"] / event_data["spy_etf"].shift(1)).fillna(0)

estimation_data['Market_Return'] = estimation_data['spy_etf'].pct_change().fillna(0)

In [121]:
estimation_data = estimation_data.dropna()

In [122]:
etf_list

['esgu_etf',
 'esgv_etf',
 'vde_etf',
 'sdus_etf',
 'tan_etf',
 'evus_etf',
 'spy_etf']

In [108]:
# 결과 저장
results = []


# 각 ETF별 동일한 이벤트 날짜 적용하여 분석
for etf in etf_list[0:6]:
    print(f"Analyzing {etf} with SPY on event date {event_date}...")

    event_data[f"{etf}_Return"] = np.log(event_data[f"{etf}"] / event_data[f"{etf}"].shift(1)).fillna(0)
    
    # 수익률 계산
    estimation_data[f"{etf}_Return"] = estimation_data[f'{etf}'].pct_change().fillna(0)
    
    # 회귀 분석 (시장 모델)
    X = sm.add_constant(estimation_data["Market_Return"])
    y = estimation_data[f"{etf}_Return"]
    model = sm.OLS(y, X).fit()
    alpha, beta = model.params

    # 비정상 수익률 (AR) 및 CAR 계산
    event_data["Expected_Return"] = alpha + beta * event_data["Market_Return"]
    a = event_data[f'{etf}_Return'].values
    b = event_data['Expected_Return'].values
        
    event_data["Abnormal_Return"] = a - b
    event_data["CAR"] = event_data["Abnormal_Return"].cumsum()

    # 결과 저장
    for date, row in event_data.iterrows():
        results.append({
            "Date": date,
            "ETF": etf,
            "Market": 'SPY',
            "Event_Date": event_date,
            "Stock_Return": row[f"{etf}_Return"],
            "Expected_Return": row["Expected_Return"],
            "Abnormal_Return": row["Abnormal_Return"],
            "CAR": row["CAR"]
        })



Analyzing esgu_etf with SPY on event date 2024-01-20 00:00:00...
Analyzing esgv_etf with SPY on event date 2024-01-20 00:00:00...
Analyzing vde_etf with SPY on event date 2024-01-20 00:00:00...
Analyzing sdus_etf with SPY on event date 2024-01-20 00:00:00...
Analyzing tan_etf with SPY on event date 2024-01-20 00:00:00...
Analyzing evus_etf with SPY on event date 2024-01-20 00:00:00...


In [109]:
df_results = pd.DataFrame(results)

In [112]:
df_results

,Date,ETF,Market,Event_Date,Stock_Return,Expected_Return,Abnormal_Return,CAR
0,2024-01-10,esgu_etf,SPY,2024-01-20,0.000000,0.000002,-0.000002,-0.000002
1,2024-01-11,esgu_etf,SPY,2024-01-20,-0.000191,-0.000431,0.000240,0.000239
2,2024-01-12,esgu_etf,SPY,2024-01-20,0.000857,0.000681,0.000176,0.000415
3,2024-01-16,esgu_etf,SPY,2024-01-20,-0.003433,-0.003608,0.000175,0.000590
4,2024-01-17,esgu_etf,SPY,2024-01-20,-0.005364,-0.005470,0.000106,0.000695
...,...,...,...,...,...,...,...,...
79,2024-01-24,evus_etf,SPY,2024-01-20,-0.005223,NaN,NaN,NaN
80,2024-01-25,evus_etf,SPY,2024-01-20,0.009219,NaN,NaN,NaN
81,2024-01-26,evus_etf,SPY,2024-01-20,-0.000116,NaN,NaN,NaN
82,2024-01-29,evus_etf,SPY,2024-01-20,0.003325,NaN,NaN,NaN


In [123]:
event_data

,esgu_etf,esgv_etf,vde_etf,sdus_etf,tan_etf,evus_etf,spy_etf,Market_Return
Date,,,,,,,,
2024-01-10,103.677231,84.219292,110.364235,8.858148,48.022583,25.143604,470.524414,0.000000
2024-01-11,103.657463,84.189606,110.480507,8.825949,46.778732,25.089697,470.317108,-0.000441
2024-01-12,103.746368,84.169838,111.943619,8.892823,46.221489,25.098518,470.642914,0.000692
2024-01-16,103.390793,83.981934,109.298378,8.886879,44.052212,24.934826,468.915100,-0.003678
2024-01-17,102.837692,83.527008,108.406944,8.820995,42.679001,24.794661,466.308533,-0.005574
2024-01-18,103.706856,84.288506,108.300354,8.873009,43.057133,24.879936,470.455322,0.008853
2024-01-19,104.990845,85.445587,108.571670,8.957222,42.688953,25.087734,476.320099,0.012389
2024-01-22,105.257523,85.801605,109.007683,9.067193,43.345707,25.169090,477.327179,0.002112
2024-01-23,105.514313,86.019180,109.337135,9.038956,44.669163,25.215160,478.719299,0.002912


In [124]:
estimation_data

,esgu_etf,esgv_etf,vde_etf,sdus_etf,tan_etf,evus_etf,spy_etf,Market_Return
Date,,,,,,,,
2024-01-10,103.677231,84.219292,110.364235,8.858148,48.022583,25.143604,470.524414,0.000000
2024-01-11,103.657463,84.189606,110.480507,8.825949,46.778732,25.089697,470.317108,-0.000441
2024-01-12,103.746368,84.169838,111.943619,8.892823,46.221489,25.098518,470.642914,0.000693
2024-01-16,103.390793,83.981934,109.298378,8.886879,44.052212,24.934826,468.915100,-0.003671
2024-01-17,102.837692,83.527008,108.406944,8.820995,42.679001,24.794661,466.308533,-0.005559
2024-01-18,103.706856,84.288506,108.300354,8.873009,43.057133,24.879936,470.455322,0.008893
2024-01-19,104.990845,85.445587,108.571670,8.957222,42.688953,25.087734,476.320099,0.012466
2024-01-22,105.257523,85.801605,109.007683,9.067193,43.345707,25.169090,477.327179,0.002114
2024-01-23,105.514313,86.019180,109.337135,9.038956,44.669163,25.215160,478.719299,0.002916


**실패**

-> 하나씩 계산하여 각 데이터프레임 결합하기